# Multi-Layer Perceptron
- 학자들은 비선형적인 모델을 만들기 위해 다음과 같은 형태의 함수를 제안했습니다
$$
f(x_i) = W_2\textrm{ReLU}(W_1x_i^T + b_1^T) + b_2.
$$

- 여기서 새롭게 제안된 parameter들의 차원은 다음과 같습니다
    - $W_2 \in \mathbb{R}^{d' \times d}$
    - $W_1 \in \mathbb{R}^{d’}$
    - $b_1 \in \mathbb{R}^{d'}$
    - $b_2 \in \mathbb{R}$($d'$은 우리가 임의로 정하는 값입니다) 

- 새롭게 제안된 함수의 형태를 자세히 살펴보시면 linear regression 모델에서 사용하던 1차 함수 $Wx_i^T + b$를 두 번 쌓은 형태와 유사하다는 것을 알 수 있습니다. 
- 차이점은 중간의 $\textrm{ReLU}(\cdot)$ 함수입니다. 
- $\textrm{ReLU}(\cdot)$는 다음과 같이 정의합니다:
$$
\textrm{ReLU}(x) = \begin{cases} x & x > 0, \\ 0 & \textrm{otherwise}.\end{cases}
$$

- 이렇게 linear regression 모델을 여러 개 쌓으면서 ReLU를 추가한 모델을 multi-layer perceptron(MLP)이라고 부릅니다.
- MLP는 다음과 같은 장점들이 있습니다
    1. **비선형 data 처리 가능:** 중간에 비선형 함수인 ReLU를 추가함으로써 비선형 data를 처리할 수 있게 됩니다.
    2. **복잡한 형태의 data 처리 가능:** linear regression 모델 하나가 아니라 여러 개를 쌓음으로써 훨씬 복잡한 형태의 data도 처리할 수 있는 expressivity(표현력)을 가집니다.

In [1]:
import torch
import numpy as np
import random

seed = 7777
random.seed(seed)

np.random.seed(seed)
torch.manual_seed(seed)
if torch.backends.mps.is_available():  # for Apple Silicon
    torch.mps.manual_seed(seed)
elif torch.cuda.is_available():  # for CUDA
    torch.cuda.manual_seed_all(seed)
else:
    pass
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
# XOR data
x = torch.tensor(
    [
        [0.0, 0.0],
        [0.0, 1.0],
        [1.0, 0.0],
        [1.0, 1.0],
    ]
)
y = torch.tensor([0, 1, 1, 0])
print(x.shape, y.shape)

torch.Size([4, 2]) torch.Size([4])


- PyTorch에서는 우리가 학습하고자 하는 함수 $f$를 보통 `torch.nn.Module` class 를 상속받은 class 로 구현합니다.
- `torch.nn.Module` class 는 abstract class 로, `def forward`를 구현하도록 abstractmethod 를 제공합니다. 이 method는 $f(x)$, 즉 함수의 출력에 해당됩니다.
- `PyTorch` 에서는 선형함수를 `torch.nn.Linear` class로 구현할 수 있습니다.
- 마찬가지로 `ReLU` 도 `torch.nn.ReLU`로 제공하고 있습니다.
    - ReLU (Rectified Linear Unit) 활성화 함수는 요소별(element-wise) 연산입니다.
    - 이는 입력 텐서의 각 원소에 독립적으로 적용되며, 텐서의 형상(shape)을 변경하지 않습니다.

위의 점들을 활용하여 2-layer MLP 구현해 봅니다.

In [3]:
from torch import nn


class Model(nn.Module):
    def __init__(self, dim, dim_prime):
        super(Model, self).__init__()
        self.layer1 = nn.Linear(dim, dim_prime)
        self.layer2 = nn.Linear(dim_prime, 1)
        self.act = nn.ReLU()

    def forward(self, x):
        # x: (n, d)
        x = self.layer1(x)  # (n, d_prime)
        x = self.act(x)  # (n, d_prime)
        x = self.layer2(x)  # (n, 1)
        return x

In [4]:
model = Model(2, 10)

- PyTorch 는 다양한 update 알고리즘들을 torch.optim 에서 제공합니다.
- gradient descent 종류인 SGD 를 import 합니다.
- `SGD`는 첫 번째 인자로 업데이트를 할 parameter들의 list를 받습니다. 예를 들어 선형 함수에서의 $w, b$가 있습니다.
- PyTorch의 `nn.Module` class는 이러한 것들을 잘 정의해주기 때문에 `model.parameters()`의 형식으로 넘겨주기만 하면 됩니다.
- 두 번째 인자는 learning rate로, 이전의 gradient descent에서 사용하던 learning rate와 똑같은 역할을 가지고 있습니다.

In [5]:
from torch.optim import SGD

optimizer = SGD(model.parameters(), lr=0.1)

In [6]:
def train(n_epochs, model, optimizer, x, y):
    for epoch in range(n_epochs):
        model.zero_grad()  # 각 parameter 의 gradient 값이 저장되어 있을 수도 있기 때문에 지워줌

        y_pred = model(x)
        loss = (y_pred[:, 0] - y).pow(2).mean()  # MSE
        loss.backward()  # model 에 있는 모든 parameter 들의 loss 에 대한 gradient 를 계산
        optimizer.step()  # 마지막으로 계산한 gradient 들을 가지고 parameter 들을 update

        print("Epoch", epoch, "| Loss:", loss.item())
    return model

In [7]:
n_epochs = 100
model = train(n_epochs, model, optimizer, x, y)

Epoch 0 | Loss: 0.7031159996986389
Epoch 1 | Loss: 0.3746700882911682
Epoch 2 | Loss: 0.28557640314102173
Epoch 3 | Loss: 0.2580499053001404
Epoch 4 | Loss: 0.24765610694885254
Epoch 5 | Loss: 0.2419496476650238
Epoch 6 | Loss: 0.2375085949897766
Epoch 7 | Loss: 0.2334129810333252
Epoch 8 | Loss: 0.2302556037902832
Epoch 9 | Loss: 0.22699156403541565
Epoch 10 | Loss: 0.22379432618618011
Epoch 11 | Loss: 0.22100991010665894
Epoch 12 | Loss: 0.21813759207725525
Epoch 13 | Loss: 0.21524953842163086
Epoch 14 | Loss: 0.212436705827713
Epoch 15 | Loss: 0.21015748381614685
Epoch 16 | Loss: 0.20739459991455078
Epoch 17 | Loss: 0.20466530323028564
Epoch 18 | Loss: 0.20203252136707306
Epoch 19 | Loss: 0.1998952031135559
Epoch 20 | Loss: 0.1972479224205017
Epoch 21 | Loss: 0.19462470710277557
Epoch 22 | Loss: 0.19202420115470886
Epoch 23 | Loss: 0.1900540143251419
Epoch 24 | Loss: 0.18754437565803528
Epoch 25 | Loss: 0.18509270250797272
Epoch 26 | Loss: 0.18265295028686523
Epoch 27 | Loss: 0.1805

In [8]:
print(model(x))
print(y)

tensor([[0.0792],
        [0.9372],
        [0.6231],
        [0.2702]], grad_fn=<AddmmBackward0>)
tensor([0, 1, 1, 0])
